In [1]:
import json
import pandas as pd
from datetime import datetime, timezone

## Erster Blick auf die Struktur der bookmarks.json Datei

In [ ]:
# Pfad zur bookmarks.json-Datei
file_path = 'example_bookmarks.json'

# Datei öffnen und JSON-Daten laden
with open(file_path, 'r', encoding='utf-8') as file:
    bookmarks_data = json.load(file)

# JSON-Daten anzeigen
print(json.dumps(bookmarks_data, indent=4, ensure_ascii=False))

## Bearbeiten der bookmarks.json

In [ ]:
# Funktion zur Umrechnung von Mikrosekunden (Unix-Epoch) in datetime
def convert_timestamp(timestamp):
    # Umrechnung von Mikrosekunden in Sekunden
    timestamp_in_seconds = timestamp / 1_000_000
    # Erstelle ein datetime-Objekt mit UTC-Zeitzone
    return datetime.fromtimestamp(timestamp_in_seconds, tz=timezone.utc)

# Funktion zur rekursiven Extraktion der Lesezeichen, einschließlich der Ordnerstruktur
def extract_bookmarks_with_folder(bookmark_data, folder_path=''):
    bookmarks_list = []
    if isinstance(bookmark_data, list):
        # Wenn die Daten eine Liste sind (Ordner oder Lesezeichen)
        for bookmark in bookmark_data:
            bookmarks_list.extend(extract_bookmarks_with_folder(bookmark, folder_path))  # Rekursion
    elif isinstance(bookmark_data, dict):
        # Wenn es ein Dictionary ist (z.B. ein Ordner oder Lesezeichen)
        if 'uri' in bookmark_data:  # Lesezeichen haben 'uri' und 'title'
            # Den vollständigen Ordnerpfad zum Lesezeichen hinzufügen
            full_folder_path = folder_path.strip(' > ')  # Entfernen des letzten " > "
            # Umwandeln der Zeitstempel in DateTime
            date_added = convert_timestamp(bookmark_data.get('dateAdded', 0))
            last_modified = convert_timestamp(bookmark_data.get('lastModified', 0))
            # Alle gewünschten Felder ins Dictionary aufnehmen
            bookmarks_list.append({
                'Title': bookmark_data['title'],
                'URL': bookmark_data['uri'],
                'Icon': bookmark_data.get('iconUri', ''),  # Optionales Icon-Feld
                'Date Added': date_added,
                'Last Modified': last_modified,
                'Folder Path': full_folder_path  # Der Ordnerpfad
            })
        # Falls es 'children' gibt, gehe weiter rekursiv durch die Unterordner
        if 'children' in bookmark_data:
            # Füge den aktuellen Ordner zur Ordnerstruktur hinzu und gehe rekursiv weiter
            new_folder_path = f"{folder_path} > {bookmark_data['title']}" if bookmark_data['title'] else folder_path
            bookmarks_list.extend(extract_bookmarks_with_folder(bookmark_data['children'], new_folder_path))
    return bookmarks_list

# Pfad zur JSON-Datei
file_path = 'example_bookmarks.json'

# Datei öffnen und JSON-Daten laden
with open(file_path, 'r', encoding='utf-8') as file:
    bookmarks_data = json.load(file)

# Rekursive Extraktion der Lesezeichen mit Ordnerpfad
bookmarks_list = extract_bookmarks_with_folder(bookmarks_data)

# Überprüfen, ob Lesezeichen extrahiert wurden
if bookmarks_list:
    # Erstelle das DataFrame
    df = pd.DataFrame(bookmarks_list)
    print(f'DataFrame erfolgreich erstellt!')
else:
    print("Keine Lesezeichen gefunden!")

In [ ]:
# Erstellten DataFrame anzeigen
df

In [5]:
# Entfernen der Zeitzoneninformationen von den Datumswerten
df['Date Added'] = df['Date Added'].dt.tz_localize(None)
df['Last Modified'] = df['Last Modified'].dt.tz_localize(None)

## DataFrame in einer Excel-Datei speichern

In [ ]:
# Exportiere das DataFrame nach Excel
output_file_path = 'example_bookmarks.xlsx'
df.to_excel(output_file_path, index=False)  # index=False, um die Index-Spalte nicht mitzuspeichern

if output_file_path:
    print(f'DataFrame erfolgreich nach {output_file_path} exportiert!')
else:
    print("Keine Lesezeichen gefunden!")